`splatlog.setup` Notes
==============================================================================

`splatlog.setup` is the function you need to call to get any logging output,
which is typically what you're looking for when you're throwing something
together quickly or jamming `splatlog` in because you need some help seeing
what's going on.

To this end, the goal of `splatlog.setup` is to be "reference-free", in the
sense that you don't need to look anything up to get what you want out of it.
That means that, ideally, whatever makes sense to write in a `setup` call should
_just work_ if reasonably possible.

That aside, `setup` should be documented, and it should be documented well. At
the moment, that's not the case.

This notebook is dedicated to:

1.  Fiddling around when writing the `splatlog.setup` doc.
2.  Exploring and cataloging the ways it "makes sense" to me to call `setup`,
    to see if we can support them.

In [1]:
import splatlog
from collections.abc import Mapping
from typing import Any
from rich.console import Console
import rich.logging

# Before anything else fix the bad ANSI blue
splatlog.setup(
    theme=splatlog.lib.rich.override_ansi_colors(
        blue="#509dea", bright_blue="#439af4"
    )
)

LOG = splatlog.getLogger(__name__)

splatlog.setup(level="info", console="stdout")

LOG.info("Hey!")

INFO      __main__                                                                                                 
msg       Hey!

The default output is `STDOUT`.

In [2]:
h = splatlog.cast_console_handler(True)
h.console.file

In [3]:
import logging


splatlog.setup(level="info", console="debug")

# Does _not_ show
LOG.debug("Shown?")

ch = splatlog.get_named_handler("console")
assert isinstance(ch, splatlog.RichHandler)
assert ch.level == splatlog.DEBUG
LOG.info(
    "levels",
    root_handler=logging.getLevelName(logging.getLogger().level),
    console_handler=ch.get_level_name(),
)

INFO      __main__                                                                                                 
msg       levels                                                                                                   
data      root_handler    str          INFO                                                                        
          console_handler str          DEBUG

In [4]:
splatlog.setup(level=splatlog.NOTSET, console="info")

LOG.info("shown?")
LOG.debug("shown?")

INFO      __main__                                                                                                 
msg       shown?

In [5]:
logging.getLevelNamesMapping()

{'CRITICAL': 50,
 'FATAL': 50,
 'ERROR': 40,
 'WARN': 30,
 'WARNING': 30,
 'INFO': 20,
 'DEBUG': 10,
 'NOTSET': 0}

In [6]:
logging.getLevelName("hey")

'Level hey'

Custom Named Handler
------------------------------------------------------------------------------

In [11]:
@splatlog.named_handler("custom", on_conflict="replace")
def cast_custom_handler(value: Any) -> logging.Handler | None:
    LOG.info("casting custom handler...", value=value)
    if value is None or value is False:
        return None

    if isinstance(value, Mapping):
        handler = rich.logging.RichHandler(**value)
        LOG.info("Constructed from mapping", handler=handler)
        return handler

    if splatlog.is_level(value):
        handler = rich.logging.RichHandler(level=value)
        LOG.info(
            "Constructed from level",
            handler=handler,
            handler_level=logging.getLevelName(handler.level),
        )
        return handler

    if isinstance(value, Console):
        handler = rich.logging.RichHandler(console=value)
        LOG.info(f"Constructed from {Console!r}", handler=handler)
        return handler

    raise ValueError(f"unexpected value: {value!r}")


splatlog.setup(custom=False)

LOG.info("custom?")


INFO      __main__.cast_custom_handler                                                                             
msg       casting custom handler...                                                                                
data      value       bool          False

[12/05/25 17:48:29] INFO     casting custom handler...                                              ]8;id=163372;file:///private/tmp/ipykernel_17158/3419792466.py\3419792466.py]8;;\:]8;id=802725;file:///private/tmp/ipykernel_17158/3419792466.py#3\3]8;;\

INFO      __main__                                                                                                 
msg       custom?